### Problem Descripition 

In 2012, URL shortening service Bitly partnered with the US government website USA.gov to provide a feed of anonymous data gathered from users who shorten links ending with .gov or .mil.

The text file comes in JSON format and here are some keys and their description. They are only the most important ones for this task.

|key| description |
|---|-----------|
| a|Denotes information about the web browser and operating system|
| tz | time zone |
| r | URL the user come from |
| u | URL where the user headed to |
| t | Timestamp when the user start using the website in UNIX format |
| hc | Timestamp when user exit the website in UNIX format |
| cy | City from which the request intiated |
| ll | Longitude and Latitude |

In the cell, I tried to provide some helper code for better understanding and clearer vision

-**HINT**- Those lines of code may be not helping at all with your task.

In [87]:
# I will try to retrieve one instance of the file in a list of dictionaries
import json
records = [json.loads(line) for line in open('data/usa.gov_click_data_1.json')]
# Print the first occurance
records[0]

{'a': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.78 Safari/535.11',
 'c': 'US',
 'nk': 1,
 'tz': 'America/New_York',
 'gr': 'MA',
 'g': 'A6qOVH',
 'h': 'wfLQtf',
 'l': 'orofrog',
 'al': 'en-US,en;q=0.8',
 'hh': '1.usa.gov',
 'r': 'http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf',
 'u': 'http://www.ncbi.nlm.nih.gov/pubmed/22415991',
 't': 1333307030,
 'hc': 1333307037,
 'cy': 'Danvers',
 'll': [42.576698, -70.954903]}

## Required

Write a script can transform the JSON files to a DataFrame and commit each file to a sparete CSV file in the target directory and consider the following:

        

All CSV files must have the following columns
- web_browser
        The web browser that has requested the service
- operating_sys
        operating system that intiated this request
- from_url

        The main URL the user came from

    **note**:

    If the retrived URL was in a long format `http://www.facebook.com/l/7AQEFzjSi/1.usa.gov/wfLQtf`

     make it appear in the file in a short format like this `www.facebook.com`
     
    
- to_url

       The same applied like `to_url`
   
- city

        The city from which the the request was sent
    
- longitude

        The longitude where the request was sent
- latitude

        The latitude where the request was sent

- time_zone
        
        The time zone that the city follow
        
- time_in

        Time when the request started
- time_out
        
        Time when the request is ended
        
        
**NOTE** :

Because that some instances of the file are incomplete, you may encouter some NaN values in your transforamtion. Make sure that the final dataframes have no NaNs at all.

### Script Details

The Script itself must do the following before and after trasforamtion: 
    
- One positional argument which is the directory path with that have the files.


- One optional argument **-u**. If this argument is passed will maintain the UNIX format of timpe stamp and if not                passed the time stamps will be converted.


- Check if the files have any dublicates in between **checksum** and print a messeage that indicate that.


- Print a message after converting each file with the number of rows transformed and the path of this file


- At the end of this script print the total excution time.
    

In [105]:
import json
import pandas as pd
from urllib.parse import urlparse
import os
import time
import hashlib

def extract_base_url(url):
    if isinstance(url, str):
        parsed_url = urlparse(url)
        return parsed_url.netloc
    else:
        return "Not Available"

def calculate_checksum(row):
    # Convert pandas Series to dictionary
    data_dict = row.to_dict()
    return hashlib.md5(json.dumps(data_dict, sort_keys=True).encode()).hexdigest()

def extract_browser_and_os(user_agent):
    parts = user_agent.split('/')
    if len(parts) >= 2:
        if len(parts) >= 3 and parts[-2] != parts[1]:
            web_browser = parts[-2].strip()
        else:
            web_browser = "Not Available"
        os_parts = parts[1].split('(')
        if len(os_parts) > 1:
            operating_system = os_parts[1].split(')')[0].strip()
        else:
            operating_system = "Not Available"
        return operating_system, web_browser
    else:
        return None, None

def process_json_files(directory_path, unix_format=False):
    start_time = time.time()
    total_rows_transformed = 0

    # Initialize an empty list to store DataFrames
    dfs = []

    # Iterate over JSON files in the directory
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r') as f:
                data = [json.loads(line) for line in f]
                df = pd.DataFrame(data)
            
            # Create checksum column
            df['checksum'] = df.apply(calculate_checksum, axis=1)
            
            # Check for duplicates based on checksum
            if df.duplicated(subset='checksum').any():
                print(f"Warning: Duplicate entries found in file '{file_path}' based on checksum.")
            
            
            #
# Check if columns 'kw' and '_heartbeat_' exist before dropping them
            if 'kw' in df.columns and '_heartbeat_' in df.columns:
                df.drop(columns=['kw', '_heartbeat_'], inplace=True)
            # Drop rows with missing values
            df.dropna(inplace=True)
            
            # Convert timestamps if needed
            if not unix_format:
                df['time_in'] = pd.to_datetime(df['time_in'], unit='s')
                df['time_out'] = pd.to_datetime(df['time_out'], unit='s')
            
            # Count number of rows transformed
            rows_transformed = len(df)
            total_rows_transformed += rows_transformed
            print(f"File '{file_path}' transformed. Rows transformed: {rows_transformed}")
            
            # Append the processed DataFrame to the list
            dfs.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(dfs, ignore_index=True)

    
    # Apply function to extract base URL
    combined_df['u'] = combined_df['u'].apply(extract_base_url)
    combined_df['r'] = combined_df['r'].apply(extract_base_url)

    # Rename columns
    combined_df.rename(columns={'cy': 'city', 'll': 'latitude_longitude', 'tz': 'time_zone', 
                        't': 'time_in', 'hc': 'time_out', 'r': 'url_incoming', 'u': 'url_outgoing', 'c': 'country_code'}, inplace=True)

    combined_df[['operating_system', 'web_browser']] = combined_df['a'].apply(lambda x: pd.Series(extract_browser_and_os(x)))

    # List of columns to keep in the desired order
    columns_to_keep = ['operating_system', 'web_browser', 'country_code', 'time_zone', 'url_incoming', 'url_outgoing', 'time_in', 'time_out', 'city', 'latitude_longitude', 'checksum']

    #Keep only the specified columns in the desired order
    combined_df = combined_df.loc[:, columns_to_keep]

    # Write DataFrame to CSV
    combined_df.to_csv('Output/combined_data.csv', index=False)

    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds")
    print(f"Total rows transformed across all files: {total_rows_transformed}")

# Example usage:
directory_path = 'Data'
unix_format = True  # Set to True to maintain UNIX format for timestamps

process_json_files(directory_path, unix_format)


File 'Data\usa.gov_click_data_1.json' transformed. Rows transformed: 8
File 'Data\usa.gov_click_data_2.json' transformed. Rows transformed: 2587
File 'Data\usa.gov_click_data_3.json' transformed. Rows transformed: 2587
Total execution time: 0.87 seconds
Total rows transformed across all files: 5182
